In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import numpy as np

In [4]:
datos = pd.DataFrame(columns=[
    'Marca', 'Modelo', 'Fecha Matriculación', 'Kilometros', 'Carburante', 'Transmisión',
    'Potencia CV', 'Potencia kW', 'Tracción', 'Tipo', 'Puertas', 'Asientos', 'Color',
    'Tapicería', 'Rueda', 'Motor Original', 'Cilindrada', 'Consumo Ciudad', 'Consumo Combinado',
    'Consumo Fuera', 'Eficiencia', 'Emisiones', 'País de Origen', 'Número de llaves', 'Coche accidentado y reparado',
    'Fecha Última Revisión', 'Tipo IVA', 'ITV válida hasta', 'Matrícula', 'Número Inventario'
])

# Inicializa el navegador
driver = webdriver.Chrome(executable_path='C:\Windows\System32\chromedriver.exe')

# Maximiza la ventana del navegador (abre en pantalla completa)
driver.maximize_window()

# Abre la página web
driver.get("https://www.autohero.com/es/search/")

# Lista para almacenar los enlaces
links = []

# Simula hacer scroll y carga los enlaces en cada scroll
for _ in range(100):  # Ajusta el número de scrolls según sea necesario
    # Hace scroll hasta el final de la página
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Espera a que la página se cargue después del scroll
    time.sleep(2)
    # Extrae los enlaces después de cada scroll y agrega a la lista
    enlaces = driver.find_elements(By.CSS_SELECTOR, 'a.link___2Maxt')
    for enlace in enlaces:
        links.append(enlace.get_attribute('href'))

# Elimina duplicados de la lista de enlaces
links = list(set(links))

# Cierra el navegador
driver.quit()

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'Se ha forzado la interrupción de una conexión existente por el host remoto', None, 10054, None))

In [5]:
#Crea el dataframe con los datos de cada link
datos = pd.DataFrame()
for pagina in links:
    #si hacemos muchas consultas a la página nos echa así que vamos a poner una espera
    time.sleep(2)
    url = pagina
    r = requests.get(url)
    html_contents = r.text
    html_soup = BeautifulSoup(html_contents, 'html.parser')
    
    #Estos valores siempre aparecen en cada página
    tit = html_soup.select_one('.titleText___edZSG')
    nombre = tit.text
    marca = nombre.split(' ', 1)[0]
    modelo = nombre.split(' ', 1)[1]
    precio = float(html_soup.select_one('.vehiclePrice___1uUmJ').text.strip('\xa0€').replace('.',''))
    version = html_soup.select_one('.subtitleText___2wcYx').text
    listaValores = [marca, modelo, precio, version]
    listaNombres = ['Marca', 'Modelo', 'Precio', 'Versión']
    
    #El resto de valores pueden o no aparecer en cada página
    #primero tratamos las etiquetas grandes de características
    carac1 = html_soup.select('.listItemValue___1IWSE')
    caracteristicas1 = [i.text for i in carac1]
    nombrecarac1 = html_soup.select('.listItemTitle___2CQBv')
    nombrecaracteristicas1 = [i.text for i in nombrecarac1]
    listaValores.extend(caracteristicas1)
    listaNombres.extend(nombrecaracteristicas1)
    
    #ahora se extrae la información de las tablas inferiores
    carac2 =html_soup.select('.body___2uId6')
    caracteristicas2 = [i.text for i in carac2]
    listaValores.extend(caracteristicas2)
    listaVal = [listaValores]
    nombrecarac2 =html_soup.select('.itemTitle___3GH8k')
    nombrecaracteristicas2 = [i.text for i in nombrecarac2]
    listaNombres.extend(nombrecaracteristicas2)
    datos_pagina = pd.DataFrame(listaVal,columns=listaNombres)
    
    datos = pd.concat([datos, datos_pagina], ignore_index=True)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [7]:
datos.to_excel('coches1.xlsx',index=False)

In [87]:
lista_datos = []
for pagina in links:
    url = pagina
    r = requests.get(url)
    html_contents = r.text
    html_soup = BeautifulSoup(html_contents, 'html.parser')
    
    tit = html_soup.select_one('.titleText___edZSG')
    nombre = tit.text
    marca = nombre.split(' ', 1)[0]
    modelo = nombre.split(' ', 1)[1]
    carac1 = html_soup.select('.listItemValue___1IWSE')
    caracteristicas1 = [i.text for i in carac1]
    carac2 =html_soup.select('.body___2uId6')
    caracteristicas2 = [i.text for i in carac2]
    if len(caracteristicas2) == 21:
        print(url)
        fechaMat = datetime.strptime(caracteristicas1[0], '%d.%m.%Y').date()
        kilometros = int(caracteristicas1[1].replace(' km', '').replace('.',''))
        carburante = caracteristicas1[2]
        transmision = caracteristicas1[3]
        potencia_cv = int(caracteristicas1[4].split('/')[0].split(' ')[0])
        potencia_kw = int(caracteristicas1[4].split('/')[1].split(' ')[1])
        traccion = caracteristicas1[5]

        tipo = caracteristicas2[0]
        puertas = int(caracteristicas2[1])
        asientos = int(caracteristicas2[2])
        color = caracteristicas2[3]
        tapiceria = caracteristicas2[4]
        rueda = caracteristicas2[5]
        motorOriginal = caracteristicas2[6]
        cilindrada = caracteristicas2[7]
        consumo = caracteristicas2[8]
        #consumoCombinado = float(caracteristicas2[8].split(')')[1].split(' ')[0])
        #consumoFuera = float(caracteristicas2[8].split(')')[2].split(' ')[0])
        eficiencia = caracteristicas2[9]
        emisiones = float(caracteristicas2[10].split(' ')[0])
        origen = caracteristicas2[12]
        llaves = int(caracteristicas2[13])
        accidentado = caracteristicas2[14]
        ultimaRevision = datetime.strptime(caracteristicas2[15], '%d.%m.%Y').date()
        tipoIVA = caracteristicas2[17]
        ITVhasta = datetime.strptime(caracteristicas2[18], '%d.%m.%Y').date()
        matricula = caracteristicas2[19]
        numInventario = caracteristicas2[20]

        datos_pagina = {
            'Marca': marca,
            'Modelo': modelo,
            'Fecha Matriculación': fechaMat,
            'Kilometros': kilometros,
            'Carburante': carburante,
            'Transmisión': transmision,
            'Potencia CV': potencia_cv,
            'Potencia kW': potencia_kw,
            'Tracción': traccion,
            'Tipo': tipo,
            'Puertas': puertas,
            'Asientos': asientos,
            'Color': color,
            'Tapicería': tapiceria,
            'Rueda': rueda,
            'Motor Original': motorOriginal,
            'Cilindrada': cilindrada,
            'Consumo': consumo,
            'Eficiencia': eficiencia,
            'Emisiones': emisiones,
            'País de Origen': origen,
            'Número de llaves': llaves,
            'Coche accidentado y reparado': accidentado,
            'Fecha Última Revisión': ultimaRevision,
            'Tipo IVA': tipoIVA,
            'ITV válida hasta': ITVhasta,
            'Matrícula': matricula,
            'Número Inventario': numInventario
        }

        lista_datos.append(datos_pagina)
    
datos = pd.DataFrame(lista_datos)

https://www.autohero.com/es/audi-a-4/id/1a7dc6f8-1a84-41a0-a85d-0ac9bb10b5be/
https://www.autohero.com/es/renault-captur/id/59b3d193-fc9d-4ae7-bd1c-2340a7b5ebe7/?impression_id=YjMwNzY4YTUtN2M3MS00ZjRmLWFiZWEtZjhmMGNlYWVkMTJlKzE3MTM4NzY2MjgxODE%3D
https://www.autohero.com/es/mercedes-benz-clase-a/id/9e8e22f9-6e1a-4a30-88b5-59af2de60655/?impression_id=NjcyMDIwMjktNmQ4Yi00NGFjLWFlNDUtZDFhNGU5NGQ2N2I2KzE3MTM4NzY2NjE1OTU%3D
https://www.autohero.com/es/ford-fiesta/id/c07b3246-ba31-47c7-897b-ebd6f24b3f79/?impression_id=ZDQzZWMxMmYtN2E3Yi00OTY5LWE2NzYtNmY5OTdkZjJmZjFhKzE3MTM4NzY2NTMyNjA%3D
https://www.autohero.com/es/seat-leon/id/8b68d981-36e6-4334-8ec8-1a1a931bdcd2/
https://www.autohero.com/es/toyota-yaris/id/b1c3f5db-562a-4b18-bc18-572aff0500be/
https://www.autohero.com/es/seat-ibiza/id/003e2aab-43c1-4e12-8925-20c8cebd4208/
https://www.autohero.com/es/mazda-cx-5/id/8a15e8c6-58dd-4ec2-879b-93c896ed6481/
https://www.autohero.com/es/ford-fiesta/id/c07b3246-ba31-47c7-897b-ebd6f24b3f79/?impressio

https://www.autohero.com/es/kia-sportage/id/19a1e601-2711-41f8-92a1-c327fec5318a/
https://www.autohero.com/es/peugeot-3008/id/d2b2e666-23c5-44a7-84f7-f96286549074/
https://www.autohero.com/es/fiat-panda/id/b9127f15-2f34-4f8f-9f72-ef625b911b02/
https://www.autohero.com/es/audi-q-5/id/7d76b1f9-0e1d-4458-8ff3-6cbc881a4fe5/
https://www.autohero.com/es/peugeot-3008/id/0e38d931-f7cf-4812-94a0-5ee81814ef37/
https://www.autohero.com/es/hyundai-tucson/id/cec8e143-c43b-4c7d-b9ef-1c1f12a39a1a/
https://www.autohero.com/es/lexus-ux-serie/id/7bb6c6ad-492a-4821-9dfe-cfb3b161d49f/
https://www.autohero.com/es/peugeot-3008/id/0e4187ab-5482-4f02-bff8-6f561faf405d/
https://www.autohero.com/es/volkswagen-polo/id/fa5a7f57-91fe-4e18-b3c4-d47210a49d3a/
https://www.autohero.com/es/mazda-cx-30/id/349d0454-85e6-4b7e-ac32-355807353ce8/
https://www.autohero.com/es/seat-ibiza/id/618bdaca-3177-4baa-93a3-5be29a2bdaf7/
https://www.autohero.com/es/seat-ateca/id/7683381e-0a08-40c1-b5cf-cac148471e2c/
https://www.autohero

https://www.autohero.com/es/toyota-corolla/id/667a5467-7908-4522-a0b0-16e02eb83345/
https://www.autohero.com/es/ford-fiesta/id/aa46d3fe-4e10-4b21-8f54-c31b957038c9/
https://www.autohero.com/es/seat-ibiza/id/2aedf977-66c4-4d89-84c8-7d8cce84954b/
https://www.autohero.com/es/opel-crossland-x/id/ab43d83a-cb02-416f-b53f-1aa51a4b36cb/
https://www.autohero.com/es/opel-mokka-x/id/d8ecb76b-6e2f-4a67-82e9-83678aed6827/
https://www.autohero.com/es/bmw-x-3/id/c7ea6a77-faf7-4ca4-b801-4b814266d70b/
https://www.autohero.com/es/kia-sportage/id/9ea05ef1-3e01-404f-878d-e1aa0a8c82a2/
https://www.autohero.com/es/renault-megane/id/5bb38e6d-fa4d-4219-b68c-939586a43bdc/
https://www.autohero.com/es/renault-megane/id/34abd8c5-a08f-44e2-8583-40dce86d5d1b/
https://www.autohero.com/es/citroen-c-3-aircross/id/087184d3-95be-40d7-b3a2-b3eae34d784f/
https://www.autohero.com/es/kia-stonic/id/46366b21-9e0d-4a7d-869a-bd25bdd605eb/?impression_id=MTAzZDVkYTYtMTVjMi00YWM4LWE2NGYtMDFlMjM3YzgzZmYxKzE3MTM4NzY2MzA0Mjk%3D
https

https://www.autohero.com/es/renault-captur/id/73197e3d-53d6-4795-b0f1-f186bef703a7/
https://www.autohero.com/es/seat-leon/id/899d8909-111f-4862-b870-862b1b493ac6/
https://www.autohero.com/es/smart-for-two/id/5544d985-c75d-4556-b5dc-905caa66b921/
https://www.autohero.com/es/volkswagen-polo/id/122303fc-d84b-4214-a183-a5961d5d9d0d/
https://www.autohero.com/es/nissan-juke/id/3570fe1b-56a6-4796-95f6-4877a319aebb/
https://www.autohero.com/es/toyota-c-hr/id/65715689-a01b-4df0-8498-dffea498a784/
https://www.autohero.com/es/seat-tarraco/id/c25b449e-a8eb-44fb-9fe5-d367c66511c2/?impression_id=NmY0OGE4M2UtYjFhZC00OGI5LWJmMDctOWEzYThjNjgwYTQ1KzE3MTM4NzY2NjYxMDQ%3D
https://www.autohero.com/es/dacia-sandero/id/e9369a7b-a248-4b56-95e0-edd0014d699e/
https://www.autohero.com/es/fiat-500/id/5e190c38-8f0e-4ead-8472-2cae791b65ee/
https://www.autohero.com/es/mazda-cx-3/id/013b916a-3a7a-4688-b837-743f00c37a6d/
https://www.autohero.com/es/fiat-500-l/id/c1585264-252d-4efb-8835-79fc97456be0/
https://www.autoher

AttributeError: 'NoneType' object has no attribute 'text'

In [102]:
datos = pd.DataFrame()
for pagina in links:
    url = pagina
    r = requests.get(url)
    html_contents = r.text
    html_soup = BeautifulSoup(html_contents, 'html.parser')
    
    tit = html_soup.select_one('.titleText___edZSG')
    nombre = tit.text
    marca = nombre.split(' ', 1)[0]
    modelo = nombre.split(' ', 1)[1]
    carac1 = html_soup.select('.listItemValue___1IWSE')
    caracteristicas1 = [i.text for i in carac1]
    carac2 =html_soup.select('.body___2uId6')
    caracteristicas2 = [[i.text for i in carac2]]
    print(caracteristicas2)
    nombrecarac2 =html_soup.select('.itemTitle___3GH8k')
    nombrecaracteristicas2 = [i.text for i in nombrecarac2]
    print(nombrecaracteristicas2)
    datos_pagina = pd.DataFrame(caracteristicas2,columns=nombrecaracteristicas2)
    
datos = pd.concat([datos, datos_pagina], ignore_index=True)

[['Familiar', '5', '5', 'Rojo', 'Cuero (Tapicería original)', 'Invierno', 'Sí', '1984 ccm', '7.9 l/100 km (En la ciudad)6.4 l/100 km (Combinado)5.5 l/100 km (Fuera de la ciudad)', 'EURO 6', '150 g/km', 'Ver estándar de calidad', 'España', '2', 'No', '20.02.2024', 'Ver historial de revisiones', 'IVA no deducible', '11.03.2026', '8696KKK', 'TN11869']]
['Tipo de vehículo', 'Puertas', 'Número de asientos', 'Color', 'Tapicería', 'Tipo de ruedas', 'Motor original', 'Cilindrada', 'Consumo', 'Clase de eficiencia CO2', 'Emisiones de CO2', 'Estándar de calidad', 'País de origen', 'Número de llaves', 'Coche accidentado y reparado', 'La última revisión se realizó el', 'Historial de revisiones', 'Tipo de IVA', 'ITV válida hasta', 'Matricula', 'Número de inventario']
[['Coche pequeño', '5', '5', 'Rojo', 'Tejido (Tapicería original)', 'Verano', 'Sí', '1197 ccm', 'EURO 6', '122 g/km', 'Ver estándar de calidad', 'España', '2', 'No', '28.02.2024', 'Ver historial de revisiones', 'IVA no deducible', '21.0

[['Coche pequeño', '5', '5', 'Azul', 'Cuero parcial (Tapicería original)', 'Verano', 'Sí', '1197 ccm', 'EURO 6', '119 g/km', 'Ver estándar de calidad', 'España', '2', 'No', '12.03.2024', 'Ver historial de revisiones', 'IVA no deducible', '31.03.2024', '0754JMX', 'GX94251']]
['Tipo de vehículo', 'Puertas', 'Número de asientos', 'Color', 'Tapicería', 'Tipo de ruedas', 'Motor original', 'Cilindrada', 'Clase de eficiencia CO2', 'Emisiones de CO2', 'Estándar de calidad', 'País de origen', 'Número de llaves', 'Coche accidentado y reparado', 'La última revisión se realizó el', 'Historial de revisiones', 'Tipo de IVA', 'ITV válida hasta', 'Matricula', 'Número de inventario']
[['Berlina', '4', '5', 'Gris', 'Tejido (Tapicería original)', 'Verano', 'Sí', '1598 ccm', '4.8 l/100 km (En la ciudad)4.2 l/100 km (Combinado)3.9 l/100 km (Fuera de la ciudad)', 'EURO 6', '112 g/km', 'Ver estándar de calidad', 'España', '2', 'No', '14.03.2024', 'Ver historial de revisiones', 'IVA no deducible', '21.12.2024

KeyboardInterrupt: 

In [103]:
datos

""


In [95]:
datos

""


In [82]:
lista_datos = []
for pagina in links:
    url = pagina
    r = requests.get(url)
    html_contents = r.text
    html_soup = BeautifulSoup(html_contents, 'html.parser')
    
    tit = html_soup.select_one('.titleText___edZSG')
    nombre = tit.text
    marca = nombre.split(' ', 1)[0]
    modelo = nombre.split(' ', 1)[1]
    carac1 = html_soup.select('.listItemValue___1IWSE')
    caracteristicas1 = [i.text for i in carac1]
    fechaMat = datetime.strptime(caracteristicas1[0], '%d.%m.%Y').date()
    kilometros = int(caracteristicas1[1].replace(' km', '').replace('.',''))
    carburante = caracteristicas1[2]
    transmision = caracteristicas1[3]
    potencia_cv = int(caracteristicas1[4].split('/')[0].split(' ')[0])
    potencia_kw = int(caracteristicas1[4].split('/')[1].split(' ')[1])
    traccion = caracteristicas1[5]
    carac2 =html_soup.select('.body___2uId6')
    caracteristicas2 = [i.text for i in carac2]
    tipo = caracteristicas2[0]
    puertas = int(caracteristicas2[1])
    asientos = int(caracteristicas2[2])
    color = caracteristicas2[3]
    tapiceria = caracteristicas2[4]
    rueda = caracteristicas2[5]
    motorOriginal = caracteristicas2[6]
    cilindrada = caracteristicas2[7]
    consumoCiudad = float(caracteristicas2[8].split(')')[0].split(' ')[0])
    consumoCombinado = float(caracteristicas2[8].split(')')[1].split(' ')[0])
    consumoFuera = float(caracteristicas2[8].split(')')[2].split(' ')[0])
    eficiencia = caracteristicas2[9]
    emisiones = float(caracteristicas2[10].split(' ')[0])
    origen = caracteristicas2[12]
    llaves = int(caracteristicas2[13])
    accidentado = caracteristicas2[14]
    ultimaRevision = datetime.strptime(caracteristicas2[15], '%d.%m.%Y').date()
    tipoIVA = caracteristicas2[17]
    ITVhasta = datetime.strptime(caracteristicas2[18], '%d.%m.%Y').date()
    matricula = caracteristicas2[19]
    numInventario = caracteristicas2[20]
    
    datos_pagina = {
        'Marca': marca,
        'Modelo': modelo,
        'Fecha Matriculación': fechaMat,
        'Kilometros': kilometros,
        'Carburante': carburante,
        'Transmisión': transmision,
        'Potencia CV': potencia_cv,
        'Potencia kW': potencia_kw,
        'Tracción': traccion,
        'Tipo': tipo,
        'Puertas': puertas,
        'Asientos': asientos,
        'Color': color,
        'Tapicería': tapiceria,
        'Rueda': rueda,
        'Motor Original': motorOriginal,
        'Cilindrada': cilindrada,
        'Consumo Ciudad': consumoCiudad,
        'Consumo Combinado': consumoCombinado,
        'Consumo Fuera': consumoFuera,
        'Eficiencia': eficiencia,
        'Emisiones': emisiones,
        'País de Origen': origen,
        'Número de llaves': llaves,
        'Coche accidentado y reparado': accidentado,
        'Fecha Última Revisión': ultimaRevision,
        'Tipo IVA': tipoIVA,
        'ITV válida hasta': ITVhasta,
        'Matrícula': matricula,
        'Número Inventario': numInventario
    }
    
    lista_datos.append(datos_pagina)
    
datos = pd.DataFrame(lista_datos)

https://www.autohero.com/es/subaru-xv/id/badb11dc-c522-4456-b162-bacaacdf0799/?impression_id=MzcxZGE4N2ItN2E1OC00MzY3LTgyZTAtMTI2YWYyNzEzMGVlKzE3MTM4NzIyNDk4NzE%3D
https://www.autohero.com/es/suzuki-swift/id/5c6a379e-ba80-4970-9303-21a75bbfac76/
https://www.autohero.com/es/audi-a-4/id/1a7dc6f8-1a84-41a0-a85d-0ac9bb10b5be/
https://www.autohero.com/es/kia-picanto/id/1aadb277-2d40-42f8-b2bb-abc9179c29ce/?impression_id=NGQwMTYyY2EtYzYyMy00MTU5LTlhZjMtMTVmOTQ4Y2U0YmZlKzE3MTM4NzIyODkwMzk%3D
https://www.autohero.com/es/peugeot-308/id/41a72ac1-5d52-4bc7-99b5-ae54f612cf62/


AttributeError: 'NoneType' object has no attribute 'text'

In [26]:
fecha_actual = datetime.now().date()

In [28]:
#para sacar los días hasta hoy
dias = fecha_actual-datetime.strptime(caracteristicas1[0], '%d.%m.%Y').date()
dias.days